In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

使用优化的TensorFlow运行时在Vertex AI预测上部署T5x基础。

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/vertex_endpoints/optimized_tensorflow_runtime/t5x_base_optimized_online_prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/vertex_endpoints/optimized_tensorflow_runtime/t5x_base_optimized_online_prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/vertex_endpoints/optimized_tensorflow_runtime/t5x_base_optimized_online_prediction.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>

## 概述

在这个示例中，您将学习如何将T5x基础模型部署到Vertex AI Prediction，使用优化的TensorFlow运行时容器。

您可以使用MLPerf推理Vertex Prediction基准工具，在优化的TensorFlow运行时容器上评估模型性能，以及不同的优化方法。

有关Vertex AI Prediction优化的TensorFlow运行时容器的更多信息，请参阅https://cloud.google.com/vertex-ai/docs/predictions/optimized-tensorflow-runtime。

### 目标

在本笔记本中，您将学习如何使用优化的TensorFlow运行时部署经过微调的T5x基础模型到Vertex AI预测服务。为了获得最佳性能，您可以使用NVIDIA A100 GPU。

您将执行的步骤包括：
* 学习如何在Vertex上微调T5x基础模型
* 使用优化的TensorFlow运行时容器将T5x基础模型部署到Vertex AI预测，使用不同的优化选项
* 对部署的模型进行基准测试并验证其预测结果

您可以使用Colab将经过微调的模型部署到Vertex AI预测。但为了获得可靠的基准测试结果，必须在与您的模型相同地区运行的Jupyter VM中运行本指南。

### 模型

在这个笔记本中，您使用了一个T5x基础模型。

T5是一个编码器-解码器模型，在多种任务的混合无监督和有监督任务上进行了预训练，每个任务都被转换为文本到文本的格式。通过将不同的前缀添加到与每个任务相对应的输入中，T5可以直接处理各种任务，例如，用于翻译：将英文翻译为德文：...，用于总结：总结：...。
模型可以进一步微调，以用于其未经训练的特定任务。

T5X是T5在JAX和Flax中的新的改进实现。
在模型经过微调后，可以将其导出为TensorFlow SavedModel格式，然后可以使用优化的TensorFlow运行时在Vertex AI Prediction上使用。

费用

本教程使用Google Cloud的以下可计费组件：

* Vertex AI
* Cloud Storage
* Cloud TPU（如果您选择在自己上调模型）

了解[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage定价](https://cloud.google.com/storage/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)根据您的预期使用情况生成成本估算。

### 设置您的本地开发环境

**如果您正在使用Colab或Vertex AI Workbench笔记本**，您的环境已经满足运行此笔记本的所有要求。您可以跳过这一步。

否则，请确保您的环境符合此笔记本的要求。
您需要以下内容：

* Google Cloud SDK
* Git
* Python 3
* virtualenv
* 在使用Python 3的虚拟环境中运行的Jupyter笔记本

Google Cloud指南中的[设置Python开发环境](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了满足这些要求的详细说明。以下步骤提供了一套简要的说明：

1. [安装并初始化Cloud SDK。](https://cloud.google.com/sdk/docs/)

2. [安装Python 3。](https://cloud.google.com/python/setup#installing_python)

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，请在终端shell中的命令行上运行`pip3 install jupyter`。

5. 要启动Jupyter，请在终端shell中的命令行上运行`jupyter notebook`。

6. 在Jupyter Notebook仪表板中打开此笔记本。

安装额外的软件包

安装执行此笔记本所需的软件包。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Vertex AI Workbench Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

!pip3 install {USER_FLAG} --upgrade google-cloud-aiplatform google-cloud-storage tensorflow-serving-api -q

如果您还计划运行MLPerf推理基准测试，您还需要下载和安装额外的依赖项（请参阅https://github.com/tensorflow/tpu/tree/master/models/experimental/inference/load_test＃run-the-benchmark-locally了解详情）。

In [ ]:
!pip3 install {USER_FLAG} transformers tf-models-official -q

In [ ]:
!git clone --recurse-submodules -b r1.0 https://github.com/mlcommons/inference.git

In [ ]:
!cd inference/loadgen && CFLAGS="-std=c++14 -O3" python3 setup.py bdist_wheel && pip3 install {USER_FLAG} --force-reinstall dist/mlperf_loadgen-*

In [ ]:
!git clone https://github.com/tensorflow/tpu.git

### 重新启动内核

在安装了额外的软件包之后，您必须重新启动笔记本内核，以便它可以找到这些软件包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

在你开始之前

### 设置你的谷歌云项目

**以下步骤适用于所有的笔记本环境。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当你第一次创建帐户时，将获得300美元的信用用于计算和存储成本。

1. [确保你的项目已启用计费功能](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

1. 如果你在本地运行这个笔记本，你必须安装 [Cloud SDK](https://cloud.google.com/sdk)。

1. 在下面的单元格中输入你的项目 ID。然后运行这个单元格，以确保 Cloud SDK 在这个笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter 运行以 `!` 开头的行作为 shell 命令，并将以 `$` 开头的 Python 变量插入这些命令中。

设置您的项目ID

如果您不知道您的项目ID，您可以尝试使用`gcloud`来获取您的项目ID。

In [ ]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

否则，在此处设置您的项目 ID。

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

设置您的地区

选择您计划要部署模型的地区。请注意，如果您计划在NVIDIA A100上部署模型，它只在特定地区可用：https://cloud.google.com/vertex-ai/docs/general/locations#region_considerations

In [ ]:
REGION = "[your-region]"  # @param {type:"string"}

if REGION == "[your-region]":
    REGION = "us-central1"

### 验证您的Google Cloud帐户

**如果您正在使用 Vertex AI Workbench笔记本**，您的环境已经经过验证。请跳过此步骤。

如果您正在使用Colab，请运行下面的单元格，并按提示进行oAuth身份验证。

否则，请按照以下步骤操作：

1. 在Cloud控制台中，转到[创建服务帐户密钥页面](https://console.cloud.google.com/apis/credentials/serviceaccountkey)。

2. 点击**创建服务帐户**。

3. 在**服务帐户名称**栏中输入名称，然后点击**创建**。

4. 在**授予此服务帐户对项目的访问权限**部分，单击**角色**下拉列表。在过滤框中输入"Vertex AI"，并选择**Vertex AI管理员**。在过滤框中输入"Storage Object Admin"，并选择**Storage Object Admin**。

5. 点击**创建**。下载包含您的密钥的JSON文件到您的本地环境。

6. 在下面的单元格中将您的服务帐户密钥路径输入为`GOOGLE_APPLICATION_CREDENTIALS`变量，然后运行该单元格。

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# The Vertex AI Workbench Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on  Vertex AI Workbench Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

# 调整 T5x 基础模型

在这个示例中，您使用了经过微调的 T5x 基础模型来进行英语到德语的语言翻译。

T5x 是一个基于 JAX 的模型，可以在 Google Cloud TPUs 上进行训练和微调，然后导出为 TensorFlow Saved 模型。

要微调模型，请按照 https://github.com/google-research/t5x 上的步骤在 Cloud TPU VM 上微调模型。或者，您可以使用 Vertex Training 服务进行模型的微调，参考 https://github.com/GoogleCloudPlatform/t5x-on-vertex-ai 获取描述如何进行此操作的步骤。

要导出微调后的模型，请参考 [导出为 TensorFlow Saved Model](https://github.com/google-research/t5x#exporting-as-tensorflow-saved-model) 部分。

为了本指南的目的，您可以使用位于 gs://cloud-samples-data/vertex-ai/model-deployment/models/t5x/base/ 下的可用的微调模型。

## 模型算法和权重类型

请注意，有两种模型类型，一种是使用 `float32` 权重导出的，另一种是使用 `bfloat16` 权重导出的。

`bfloat16` 是 Google Cloud TPUs 的一种原生格式，T5x 模型也默认使用它。
NVIDIA A100 GPU 支持 `bfloat16` 算法，并且经过优化的 TensorFlow 运行时可以利用这一优势。
如果您计划在不支持 `bfloat16` 的 GPU 上部署模型，比如 NVIDIA T4 或 NVIDIA V100，您需要使用带有 `float32` 权重的模型。幸运的是，经过优化的 TensorFlow 运行时具有一项优化，允许通过指定 `--allow_compression` 选项来在较低精度上运行模型。

In [ ]:
T5X_BASE_FLOAT32_MODEL_URI = "gs://cloud-samples-data/vertex-ai/model-deployment/models/t5x/base/saved_model.float32/1"
T5X_BASE_BFLOAT16_MODEL_URI = "gs://cloud-samples-data/vertex-ai/model-deployment/models/t5x/base/saved_model.bfloat16/1"

你可以使用`saved_model_cli`工具来观察 TensorFlow 的模型定义。可以忽略与`SentencepieceOp`相关的错误。

In [ ]:
!saved_model_cli show --dir=$T5X_BASE_FLOAT32_MODEL_URI --all

将模型部署到Vertex AI端点

您可以使用[Vertex AI SDK](https://cloud.google.com/python/docs/reference/aiplatform/latest)部署模型，并将其导入到您的笔记本环境中。

In [ ]:
from google.cloud import aiplatform

定义用于部署的节点配置。有关 Vertex AI 预测选项的详细信息，请参阅[配置计算资源](https://cloud.google.com/vertex-ai/docs/predictions/configure-compute)。

您将使用优化后的TensorFlow运行时容器部署模型，请查看官方文档中提供的完整容器列表：https://cloud.google.com/vertex-ai/docs/predictions/optimized-tensorflow-runtime#available_container_images。

In [ ]:
OPTIMIZED_TF_RUNTIME_IMAGE_URI = (
    "us-docker.pkg.dev/vertex-ai-restricted/prediction/tf_opt-gpu.nightly:latest"
)

您将在NVIDIA T4 GPU上部署T5x模型。

In [ ]:
DEPLOY_COMPUTE_T4 = "n1-standard-8"
DEPLOY_GPU_T4 = "NVIDIA_TESLA_T4"

部署带有float32权重和无优化的T5x基本模型。

In [ ]:
t5x_base_float32 = aiplatform.Model.upload(
    display_name="t5x_base_float32",
    artifact_uri=T5X_BASE_FLOAT32_MODEL_URI,
    serving_container_image_uri=OPTIMIZED_TF_RUNTIME_IMAGE_URI,
    serving_container_args=[],
    location=REGION,
)

t5x_base_float32_t4_endpoint = t5x_base_float32.deploy(
    deployed_model_display_name="t5x_base_float32_deployed",
    traffic_split={"0": 100},
    machine_type=DEPLOY_COMPUTE_T4,
    accelerator_type=DEPLOY_GPU_T4,
    accelerator_count=1,
    min_replica_count=1,
    max_replica_count=1,
)

部署带有float32权重和预编译的T5x基础模型。

In [ ]:
t5x_base_float32_precompiled = aiplatform.Model.upload(
    display_name="t5x_base_float32_precompiled",
    artifact_uri=T5X_BASE_FLOAT32_MODEL_URI,
    serving_container_image_uri=OPTIMIZED_TF_RUNTIME_IMAGE_URI,
    serving_container_args=["--allow_precompilation"],
    location=REGION,
)

t5x_base_float32_precompiled_t4_endpoint = t5x_base_float32_precompiled.deploy(
    deployed_model_display_name="t5x_base_float32_precompiled_deployed",
    traffic_split={"0": 100},
    machine_type=DEPLOY_COMPUTE_T4,
    accelerator_type=DEPLOY_GPU_T4,
    accelerator_count=1,
    min_replica_count=1,
    max_replica_count=1,
)

部署具有float32权重、预编译和压缩的T5x基础模型。模型压缩优化模型，使计算密集型部分以较低的float16精度运行，并利用NVIDIA GPU TensorCores。

In [ ]:
t5x_base_float32_precompiled_mixedprecision = aiplatform.Model.upload(
    display_name="t5x_base_float32_precompiled_mixedprecision",
    artifact_uri=T5X_BASE_FLOAT32_MODEL_URI,
    serving_container_image_uri=OPTIMIZED_TF_RUNTIME_IMAGE_URI,
    serving_container_args=["--allow_precompilation", "--allow_compression"],
    location=REGION,
)

t5x_base_float32_precompiled_mixedprecision_t4_endpoint = t5x_base_float32_precompiled_mixedprecision.deploy(
    deployed_model_display_name="t5x_base_float32_precompiled_mixedprecision_deployed",
    traffic_split={"0": 100},
    machine_type=DEPLOY_COMPUTE_T4,
    accelerator_type=DEPLOY_GPU_T4,
    accelerator_count=1,
    min_replica_count=1,
    max_replica_count=1,
)

为了达到最佳性能，您可以在具有对bfloat16算术支持的NVIDIA A100上部署带有bfloat16权重的T5x基本模型。请注意，为了有效利用bfloat16逻辑，模型必须通过预编译部署。由于模型已经以半精度运行，因此不需要模型压缩。

In [ ]:
DEPLOY_COMPUTE_A100 = "a2-highgpu-1g"
DEPLOY_GPU_A100 = "NVIDIA_TESLA_A100"

In [ ]:
t5x_base_float32_a100_endpoint = t5x_base_float32.deploy(
    deployed_model_display_name="t5x_base_float32_deployed",
    traffic_split={"0": 100},
    machine_type=DEPLOY_COMPUTE_A100,
    accelerator_type=DEPLOY_GPU_A100,
    accelerator_count=1,
    min_replica_count=1,
    max_replica_count=1,
)

In [ ]:
t5x_base_bfloat16_precompiled = aiplatform.Model.upload(
    display_name="t5x_base_bfloat16_precompiled",
    artifact_uri=T5X_BASE_BFLOAT16_MODEL_URI,
    serving_container_image_uri=OPTIMIZED_TF_RUNTIME_IMAGE_URI,
    serving_container_args=["--allow_precompilation"],
    location=REGION,
)

t5x_base_bfloat16_precompiled_a100_endpoint = t5x_base_bfloat16_precompiled.deploy(
    deployed_model_display_name="t5x_base_bfloat16_precompiled_deployed",
    traffic_split={"0": 100},
    machine_type=DEPLOY_COMPUTE_A100,
    accelerator_type=DEPLOY_GPU_A100,
    accelerator_count=1,
    min_replica_count=1,
    max_replica_count=1,
)

发送预测请求

您可以直接从每个端点发送请求。T5x模型期望数据是以字典形式存储的，其中包含键为"text_batch"（参见上面`saved_model_cli`调用的响应）。

In [ ]:
instances = [{"text_batch": "translate English to German: this is good"}]

In [ ]:
t5x_base_float32_t4_endpoint.predict(instances=instances)

In [ ]:
t5x_base_float32_precompiled_t4_endpoint.predict(instances=instances)

In [ ]:
t5x_base_float32_precompiled_mixedprecision_t4_endpoint.predict(instances=instances)

In [ ]:
t5x_base_float32_a100_endpoint.predict(instances=instances)

In [ ]:
t5x_base_bfloat16_precompiled_a100_endpoint.predict(instances=instances)

另外，您还可以发送 POST REST 请求而不使用 SDK。 了解更多关于 https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-custom-models#online_predict_custom_trained-drest。 这种方法稍微更快。

## 比较预测

为了确保所有模型返回相同的结果，向所有端点发送相同的请求，并比较预测结果。

In [ ]:
# Add your samples here
samples = [
    "hello world",
    "this is a prediction from T5x model",
    "this is good",
    "my name is T5x",
]

endpoints = {
    "t5x_base_float32_t4": t5x_base_float32_t4_endpoint,
    "t5x_base_float32_precompiled_t4": t5x_base_float32_precompiled_t4_endpoint,
    "t5x_base_float32_precompiled_mixedprecision_t4": t5x_base_float32_precompiled_mixedprecision_t4_endpoint,
    "t5x_base_float32_a100": t5x_base_float32_a100_endpoint,
    "t5x_base_bfloat16_precompiled_a100": t5x_base_bfloat16_precompiled_a100_endpoint,
}

prefix = "translate English to German: "

for sample in samples:
    print(f"Prediction for: {prefix}{sample}")
    for model_name, endpoint in endpoints.items():
        response = endpoint.predict(instances=[{"text_batch": f"{prefix}{sample}"}])
        prediction = response.predictions[0]["output_0"][0]
        print(f"Model: {model_name} Prediction: {prediction}")
    print("-----------")

##（可选）比较部署模型的性能

你可以在Colab环境中运行基准测试，为了获得可靠的结果，你应该使用与你的模型相同区域的虚拟机。

导入用于基准测试模型的辅助函数。

In [ ]:
!curl https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/vertex_endpoints/optimized_tensorflow_runtime/benchmark.py -o benchmark.py

In [ ]:
from benchmark import benchmark

这段代码以给定的QPS异步均匀地发送指定数量的请求，然后记录观察到的延迟。接下来，聚合延迟结果并计算百分位数。模型能够处理的实际每秒请求数（actual_qps）是指模型处理发送的请求所需的时间除以请求数。通过为 send_request 和 build_request 函数提供不同的实现，相同的代码可以用于对在本地运行或使用 gRPC 和 REST 协议在 Vertex AI Prediction 上运行的模型进行基准测试。

此基准测试的主要目标是测量模型在不同负载下的延迟，以及模型能够处理的最大吞吐量。为了找到最大吞吐量，逐渐增加 QPS 直到 actual_qps 停止增加并且延迟急剧增加。

在生产部署中，工作负载并不是均匀的，因此最大模型吞吐量可能较低。目标不是模拟生产工作负载，这个基准测试旨在比较在不同环境中运行相同模型时的延迟和吞吐量。

In [ ]:
def build_rest_request(row_dict, model_name):
    return row_dict


def validate_response(response):
    assert response
    assert len(response.predictions) == 1
    assert "output_0" in response.predictions[0]
    assert response.predictions[0]["output_0"]

In [ ]:
def send_rest_request(request):
    response = t5x_base_float32_t4_endpoint.predict(instances=[request])
    validate_response(response)


t5x_base_float32_t4_results = benchmark(
    send_rest_request,
    build_rest_request,
    "gs://cloud-samples-data/vertex-ai/model-deployment/models/t5x/requests/requests_100.jsonl",
    [0.5, 0.75, 1, 1.25, 1.5, 1.75, 2.0],
    10,
)

t5x_base_float32_t4_results

In [ ]:
def send_rest_request(request):
    response = t5x_base_float32_precompiled_t4_endpoint.predict(instances=[request])
    validate_response(response)


t5x_base_float32_precompiled_t4_results = benchmark(
    send_rest_request,
    build_rest_request,
    "gs://cloud-samples-data/vertex-ai/model-deployment/models/t5x/requests/requests_100.jsonl",
    [0.5, 1, 2, 3, 4, 5],
    10,
)

t5x_base_float32_precompiled_t4_results

In [ ]:
def send_rest_request(request):
    response = t5x_base_float32_precompiled_mixedprecision_t4_endpoint.predict(
        instances=[request]
    )
    validate_response(response)


t5x_base_float32_precompiled_mixedprecision_t4_results = benchmark(
    send_rest_request,
    build_rest_request,
    "gs://cloud-samples-data/vertex-ai/model-deployment/models/t5x/requests/requests_100.jsonl",
    [0.5, 1, 2, 3, 4, 5],
    10,
)

t5x_base_float32_precompiled_mixedprecision_t4_results

In [ ]:
def send_rest_request(request):
    response = t5x_base_float32_a100_endpoint.predict(instances=[request])
    validate_response(response)


t5x_base_float32_a100_results = benchmark(
    send_rest_request,
    build_rest_request,
    "gs://cloud-samples-data/vertex-ai/model-deployment/models/t5x/requests/requests_100.jsonl",
    [0.5, 1, 1.25, 1.5, 1.75, 2.0, 2.25],
    10,
)

t5x_base_float32_a100_results

In [ ]:
def send_rest_request(request):
    response = t5x_base_bfloat16_precompiled_a100_endpoint.predict(instances=[request])
    validate_response(response)


t5x_base_bfloat16_precompiled_a100_results = benchmark(
    send_rest_request,
    build_rest_request,
    "gs://cloud-samples-data/vertex-ai/model-deployment/models/t5x/requests/requests_100.jsonl",
    [0.5, 5, 7.5, 10, 12.5, 15],
    10,
)

t5x_base_bfloat16_precompiled_a100_results

整合并可视化结果。

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np


def build_graph(x_key, y_key, results_dict, axis, title="T5x base model latency"):
    matplotlib.rcParams["figure.figsize"] = [10.0, 7.0]

    fig, ax = plt.subplots(facecolor=(1, 1, 1))
    ax.set_xlabel("QPS")
    ax.set_ylabel("Latency(ms)")
    for label, results in results_dict.items():
        x = np.array(results[x_key])
        y = np.array(results[y_key])
        ax.plot(x, y, label=label, marker="s")
    ax.grid()
    ax.legend()
    ax.axis(axis)
    ax.set_title(title)
    return fig

In [ ]:
fig = build_graph(
    "actual_qps",
    "p50",
    {
        "T5x base float32 on T4": t5x_base_float32_t4_results,
        "T5x base float32 on T4 with precompilation": t5x_base_float32_precompiled_t4_results,
        "T5x base float32 on T4 with precompilation and compression": t5x_base_float32_precompiled_mixedprecision_t4_results,
        "T5x base float32 on A100": t5x_base_float32_a100_results,
        "T5x base bfloat16 on A100 with precompilation": t5x_base_bfloat16_precompiled_a100_results,
    },
    (0, 10, 0, 2500),
    title="T5x base model p50 latency, batch size 1",
)
fig.savefig("t5x_base_p50_latency.png", bbox_inches="tight")

In [ ]:
fig = build_graph(
    "actual_qps",
    "p99",
    {
        "T5x base float32 on T4": t5x_base_float32_t4_results,
        "T5x base float32 on T4 with precompilation": t5x_base_float32_precompiled_t4_results,
        "T5x base float32 on T4 with precompilation and compression": t5x_base_float32_precompiled_mixedprecision_t4_results,
        "T5x base float32 on A100": t5x_base_float32_a100_results,
        "T5x base bfloat16 on A100 with precompilation": t5x_base_bfloat16_precompiled_a100_results,
    },
    (0, 5, 0, 5500),
    title="T5x base model p99 latency, batch size 1",
)
fig.savefig("t5x_base_p99_latency.png", bbox_inches="tight")

正如您所看到的，Vertex AI Prediction 优化了 TensorFlow 运行时优化，为 T5x 基础模型提供了显著更高的吞吐量和更低的延迟。

## （可选）使用MLPerf推断loadgen比较部署模型的性能

MLPerf 推理是一个基准套件，用于衡量系统在各种部署场景中运行模型的速度。MLPerf 现在是衡量模型性能的行业标准方式。您可以在 https://github.com/tensorflow/tpu/tree/master/models/experimental/inference/load_test 上按照说明运行部署模型的 MLPerf 推理基准测试。

与以前使用的简单基准测试不同，MLPerf loadgen 使用 Poisson 分布发送请求。

In [ ]:
project_id = t5x_base_float32_t4_endpoint.resource_name.split("/")[1]
project_id

endpoint_id = t5x_base_float32_t4_endpoint.resource_name.split("/")[-1]
endpoint_id

In [ ]:
%cd tpu/models/experimental/inference

In [ ]:
!python3 -m load_test.examples.loadgen_vertex_main \
  --project_id={project_id} \
  --region={REGION} \
  --endpoint_id={t5x_base_float32_t4_endpoint.resource_name.split("/")[-1]} \
  --dataset=generic_jsonl \
  --data_file=gs://cloud-samples-data/vertex-ai/model-deployment/models/t5x/requests/requests_100.jsonl \
  --api_type=rest \
  --min_query_count=10 \
  --min_duration_ms=10000 \
  --qps=0.5 --qps=1.0 --qps=1.25 --qps=1.5 --qps=1.75 --qps=2.0 \
  --csv_report_filename="t5x_base_float32_t4_results.csv"

In [ ]:
!python3 -m load_test.examples.loadgen_vertex_main \
  --project_id={project_id} \
  --region={REGION} \
  --endpoint_id={t5x_base_float32_precompiled_t4_endpoint.resource_name.split("/")[-1]} \
  --dataset=generic_jsonl \
  --data_file=gs://cloud-samples-data/vertex-ai/model-deployment/models/t5x/requests/requests_100.jsonl \
  --api_type=rest \
  --min_query_count=10 \
  --min_duration_ms=10000 \
  --qps=0.5 --qps=1 --qps=2 --qps=3 --qps=4 --qps=5 \
  --csv_report_filename="t5x_base_float32_precompiled_t4_results.csv"

In [ ]:
!python3 -m load_test.examples.loadgen_vertex_main \
  --project_id={project_id} \
  --region={REGION} \
  --endpoint_id={t5x_base_float32_precompiled_mixedprecision_t4_endpoint.resource_name.split("/")[-1]} \
  --dataset=generic_jsonl \
  --data_file=gs://cloud-samples-data/vertex-ai/model-deployment/models/t5x/requests/requests_100.jsonl \
  --api_type=rest \
  --min_query_count=10 \
  --min_duration_ms=10000 \
  --qps=0.5 --qps=1 --qps=2 --qps=3 --qps=4 --qps=5 --qps=6 \
  --csv_report_filename="t5x_base_float32_precompiled_mixedprecision_t4_results.csv"

In [ ]:
!python3 -m load_test.examples.loadgen_vertex_main \
  --project_id={project_id} \
  --region={REGION} \
  --endpoint_id={t5x_base_float32_a100_endpoint.resource_name.split("/")[-1]} \
  --dataset=generic_jsonl \
  --data_file=gs://cloud-samples-data/vertex-ai/model-deployment/models/t5x/requests/requests_100.jsonl \
  --api_type=rest \
  --min_query_count=10 \
  --min_duration_ms=10000 \
  --qps=0.5 --qps=1.0 --qps=1.25 --qps=1.5 --qps=1.75 --qps=2.0 --qps=2.25 \
  --csv_report_filename="t5x_base_float32_a100_results.csv"

In [ ]:
!python3 -m load_test.examples.loadgen_vertex_main \
  --project_id={project_id} \
  --region={REGION} \
  --endpoint_id={t5x_base_bfloat16_precompiled_a100_endpoint.resource_name.split("/")[-1]} \
  --dataset=generic_jsonl \
  --data_file=gs://cloud-samples-data/vertex-ai/model-deployment/models/t5x/requests/requests_100.jsonl \
  --api_type=rest \
  --min_query_count=10 \
  --min_duration_ms=10000 \
  --qps=0.5 --qps=5 --qps=7.5 --qps=10 --qps=12.5 --qps=15 \
  --csv_report_filename="t5x_base_bfloat16_precompiled_a100_results.csv"

In [ ]:
import csv


def parse_report_csv(file_name):
    with open(file_name, newline="") as f:
        reader = csv.reader(f)

        d = {}
        index_to_key = {}
        for row in reader:
            if not d:
                for index in range(len(row)):
                    key = row[index]
                    index_to_key[index] = key
                    d[key] = []
            else:
                for index in range(len(row)):
                    if index_to_key[index] != "scenario":
                        d[index_to_key[index]].append(float(row[index]))
                    else:
                        d[index_to_key[index]].append(row[index])
        return d

In [ ]:
fig = build_graph(
    "actual_qps",
    "p50",
    {
        "T5x base float32 on T4": parse_report_csv("t5x_base_float32_t4_results.csv"),
        "T5x base float32 on T4 with precompilation": parse_report_csv(
            "t5x_base_float32_precompiled_t4_results.csv"
        ),
        "T5x base float32 on T4 with precompilation and compression": parse_report_csv(
            "t5x_base_float32_precompiled_mixedprecision_t4_results.csv"
        ),
        "T5x base float32 on A100": parse_report_csv(
            "t5x_base_float32_a100_results.csv"
        ),
        "T5x base bfloat16 on A100 with precompilation": parse_report_csv(
            "t5x_base_bfloat16_precompiled_a100_results.csv"
        ),
    },
    (0, 10, 0, 2500),
    title="T5x base model p50 latency measured by MLPerf loadgen, batch size 1",
)
fig.savefig("t5x_base_p50_mlperf_latency.png", bbox_inches="tight")

In [ ]:
fig = build_graph(
    "actual_qps",
    "p99",
    {
        "T5x base float32 on T4": parse_report_csv("t5x_base_float32_t4_results.csv"),
        "T5x base float32 on T4 with precompilation": parse_report_csv(
            "t5x_base_float32_precompiled_t4_results.csv"
        ),
        "T5x base float32 on T4 with precompilation and compression": parse_report_csv(
            "t5x_base_float32_precompiled_mixedprecision_t4_results.csv"
        ),
        "T5x base float32 on A100": parse_report_csv(
            "t5x_base_float32_a100_results.csv"
        ),
        "T5x base bfloat16 on A100 with precompilation": parse_report_csv(
            "t5x_base_bfloat16_precompiled_a100_results.csv"
        ),
    },
    (0, 10, 0, 3500),
    title="T5x base model p99 latency measured by MLPerf loadgen, batch size 1",
)
fig.savefig("t5x_base_p99_mlperf_latency.png", bbox_inches="tight")

这些结果与使用朴素基准代码获得的结果大致一致。

清理

完成后，可以安全地移除您创建的端点和部署的模型。

In [ ]:
# Undeploy models
t5x_base_float32_t4_endpoint.undeploy_all()
t5x_base_float32_precompiled_t4_endpoint.undeploy_all()
t5x_base_float32_precompiled_mixedprecision_t4_endpoint.undeploy_all()
t5x_base_float32_a100_endpoint.undeploy_all()
t5x_base_bfloat16_precompiled_a100_endpoint.undeploy_all()

In [ ]:
# Delete models
t5x_base_float32.delete()
t5x_base_float32_precompiled.delete()
t5x_base_float32_precompiled_mixedprecision.delete()
t5x_base_bfloat16_precompiled.delete()

In [ ]:
# Delete endpoints
t5x_base_float32_t4_endpoint.delete()
t5x_base_float32_precompiled_t4_endpoint.delete()
t5x_base_float32_precompiled_mixedprecision_t4_endpoint.delete()
t5x_base_float32_a100_endpoint.delete()
t5x_base_bfloat16_precompiled_a100_endpoint.delete()